In [ ]:
input_path = "."
output_path = "needs_relabel"

In [ ]:
from bs4 import BeautifulSoup
import os
import shutil
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import time
%matplotlib inline

In [ ]:
def draw_annotation_rectangle(figure, labeled_object):
    bound_box = labeled_object.bndbox
    rectangle = figure.add_subplot(111, aspect='equal')
    xmin = int(bound_box.xmin.text)
    xmax = int(bound_box.xmax.text)
    ymin = int(bound_box.ymin.text)
    ymax = int(bound_box.ymax.text)
    rectangle.add_patch(patches.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, fill=False, 
                                          edgecolor=np.random.rand(3,1)))
    return labeled_object.find('name').text

def reject_image_label(xml_path, name):
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    annotation = load_image_annotation(xml_path)
    shutil.copyfile(annotation.path.text, '{}/{}.jpg'.format(output_path, annotation.filename.text))
    
def load_image_annotation(xml_filename):
    annotation = None
    with open(xml_filename) as xml_file:
        soup = BeautifulSoup(xml_file, 'lxml')
        annotation = soup.html.body.annotation
    return annotation

def display_img(xml_filename):
    annotation = load_image_annotation(xml_filename)
    img = mpimg.imread(annotation.path.text)
    figure = plt.figure()
    labels = []
    for index, labeled_object in enumerate(annotation.find_all('object')):
        labels.append(draw_annotation_rectangle(figure, labeled_object))
    plt.legend(labels=labels)
    plt.imshow(img, cmap='Set3')
    
def get_all_xml_files(directory):
    return [filename for filename in os.listdir(directory) if ".xml" in filename]

In [ ]:
xml_files = get_all_xml_files(input_path)

In [ ]:
current_file = xml_files.pop()
display_img('{}/{}'.format(input_path, current_file))

In [ ]:
reject_image_label('{}/{}'.format(input_path, current_file), current_file)